 Semantic Similarity Search

In [1]:
pip install tensorflow_hub -q

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import os

In [3]:
import sys
print(sys.version)
tf.version

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/version/__init__.py'>

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url) #download and load the model
print("Done!")

Done!


In [6]:
# Load data
outpath = r"/content/"
infile = os.path.join(outpath, "movies_metadata.csv")

df = pd.read_csv(infile, nrows=20000)
df = df[['original_title']]
print(len(df))
df.head(6)

20000


original_title
0                    Toy Story
1                      Jumanji
2             Grumpier Old Men
3            Waiting to Exhale
4  Father of the Bride Part II
5                         Heat

In [7]:
df['original_title'] = df['original_title'].str.lower()
df

original_title
0                                              toy story
1                                                jumanji
2                                       grumpier old men
3                                      waiting to exhale
4                            father of the bride part ii
...                                                  ...
19995                               l'ordre et la morale
19996                                         versailles
19997                                    two in the wave
19998  lotte reiniger: homage to the inventor of the ...
19999  rko production 601: the making of 'kong, the e...

[20000 rows x 1 columns]

In [8]:
# creating my empty list
doc = []

# Iterate over each row
for index, rows in df.iterrows():
    # Create list for the current row
    mylist =[rows.original_title]
    doc.append(mylist[0])

# Print the list doc
print(doc)

['toy story', 'jumanji', 'grumpier old men', 'waiting to exhale', 'father of the bride part ii', 'heat', 'sabrina', 'tom and huck', 'sudden death', 'goldeneye', 'the american president', 'dracula: dead and loving it', 'balto', 'nixon', 'cutthroat island', 'casino', 'sense and sensibility', 'four rooms', 'ace ventura: when nature calls', 'money train', 'get shorty', 'copycat', 'assassins', 'powder', 'leaving las vegas', 'othello', 'now and then', 'persuasion', 'la cité des enfants perdus', '摇啊摇，摇到外婆桥', 'dangerous minds', 'twelve monkeys', 'guillaumet, les ailes du courage', 'babe', 'carrington', 'dead man walking', 'across the sea of time', 'it takes two', 'clueless', 'cry, the beloved country', 'richard iii', 'dead presidents', 'restoration', 'mortal kombat', 'to die for', 'how to make an american quilt', 'se7en', 'pocahontas', 'when night is falling', 'the usual suspects', 'guardian angel', 'mighty aphrodite', 'lamerica', 'the big green', 'georgia', 'kids of the round table', 'home fo

In [9]:
# How big is my data
print("Total number of text samples: ", len(doc))

Total number of text samples:  20000


In [10]:
# Vectorizing my data
bank_vec = model(doc)
print(bank_vec.shape)
print(bank_vec)

(20000, 512)
tf.Tensor(
[[ 0.01755411 -0.01248422 -0.02224629 ... -0.01341353  0.08217454
   0.00417209]
 [ 0.04525361  0.01810172 -0.00362508 ... -0.03219749  0.08333101
  -0.04444902]
 [ 0.03017413 -0.0265909  -0.00645986 ... -0.00954842  0.08587465
   0.03155793]
 ...
 [-0.02897701 -0.04300207 -0.02740624 ... -0.00808136 -0.02983273
   0.00389728]
 [ 0.06968129  0.05682554 -0.02684712 ...  0.02578602  0.08957793
  -0.07593115]
 [ 0.04867511 -0.05056433  0.05215009 ... -0.0735167   0.01742515
  -0.07284751]], shape=(20000, 512), dtype=float32)


In [11]:
# Vectoryzing my query
mystr = 'dogs n cats'
#mystr = (mystr).upper()
print(mystr)
query = [mystr]
query_vec = model(query) #encodes the string in the list

dogs n cats


In [12]:
# inner product calculation, recordando linear algebra
correlation = np.transpose(np.inner(query_vec,bank_vec))

In [13]:
result = str(doc[np.argmax(correlation, axis=0)[0]]).strip().format()
print(result)

cats & dogs


In [14]:
print("Closest match found to '",query[0],"' is '",
      doc[np.argmax(correlation, axis=0)[0]],"'", np.argmax(correlation, axis=0),
      max(correlation))

Closest match found to ' dogs n cats ' is ' cats & dogs ' [4258] [0.87108845]


In [15]:
top_indices = np.argsort(correlation, axis=0)[-5:][::-1].squeeze()

# Print the top five matches
print("Top five closest matches to '", query[0], "' are:")
for idx in top_indices:
    print(doc[idx], correlation[idx])

# If you want to access the top five matches individually:
top_matches = [(doc[idx], correlation[idx]) for idx in top_indices]

Top five closest matches to ' dogs n cats ' are:
cats & dogs [0.87108845]
the truth about cats & dogs [0.7565437]
cats [0.70335066]
cat people [0.6255593]
cat people [0.6255593]
